In [20]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
from dotenv import load_dotenv, find_dotenv
from llama_index.core import SimpleDirectoryReader

#load environment variables from .env file
_ = load_dotenv

In [22]:
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader("../data").load_data()
# node_parser = SentenceSplitter(chunk_size=4096)
# nodes = node_parser.get_nodes_from_documents(documents)

### qa with Ragas


In [23]:
# %pip install --upgrade "huggingface_hub>=0.20.0"

from ragas.testset import TestsetGenerator
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# generator with openai models
generator_llm = OpenAI(model="gpt-4.1-nano")
embeddings = OpenAIEmbedding(model="text-embedding-3-large")

generator = TestsetGenerator.from_llama_index(
    llm=generator_llm,
    embedding_model=embeddings,
)

In [24]:
# generate testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    testset_size=100,
)

Generating Samples: 100%|██████████| 100/100 [00:09<00:00, 10.59it/s]


In [25]:
df = testset.to_pandas()
df.tail()

,user_input,reference_contexts,reference,synthesizer_name
95,How can I show I am a trusted authority to get...,"[<1-hop>\n\n \n \n 6 \n \n \nSo, later in the...","According to the context, after listening in a...",multi_hop_specific_query_synthesizer
96,How can sales professionals establish themselv...,"[<1-hop>\n\n \n \n 6 \n \n \nSo, later in the...",Sales professionals can establish themselves a...,multi_hop_specific_query_synthesizer
97,How can a salesperson effectively use NEPQ que...,[<1-hop>\n\n \n \n 36 \n \nSalesperson: I wil...,The context illustrates that using NEPQ techni...,multi_hop_specific_query_synthesizer
98,"How can strategic questioning techniques, as d...",[<1-hop>\n\n \n \n 36 \n \nSalesperson: I wil...,The context highlights that using strategic qu...,multi_hop_specific_query_synthesizer
99,"How can strategic questioning techniques, as e...",[<1-hop>\n\n \n \n 36 \n \nSalesperson: I wil...,The context illustrates that effective strateg...,multi_hop_specific_query_synthesizer


In [26]:
df.reset_index(drop=True, inplace=True)
df.to_csv(f"./dataset.csv", index=False)

#### remove duplicate

In [28]:
import pandas as pd
dataset = pd.read_csv(f"./dataset.csv")
dataset.head()

,user_input,reference_contexts,reference,synthesizer_name
0,Can you provide detailed information about the...,[' \n \nPROSPECTBOSS \nPROSPECTBOSS SALES TEA...,The PROSPECTBOSS SALES TEAM can be contacted a...,single_hop_specifc_query_synthesizer
1,What do NEURO-EMOTIONAL PERSUATION QUESTIONS d...,"["" \n \n 2 \n \nWHAT NEPQ (NEURO-EMOTIONAL PE...",NEURO-EMOTIONAL PERSUATION QUESTIONS help gath...,single_hop_specifc_query_synthesizer
2,How do your prospects benefit from engaging in...,"["" \n \n 3 \n \nA good doctor follows a simil...",Your prospects benefit from engaging in strate...,single_hop_specifc_query_synthesizer
3,What does the concept of Level 8 imply about t...,[' \n \n 5 \n \ninitiate a conversation by as...,"Level 8 refers to the process where prospects,...",single_hop_specifc_query_synthesizer
4,How do you become a trusted authority?,"[' \n \n 6 \n \n \nSo, later in the sales con...",You become a trusted authority by listening to...,single_hop_specifc_query_synthesizer


In [30]:
df_cleaned = dataset.drop_duplicates(subset=['user_input'])
df_cleaned.head()

,user_input,reference_contexts,reference,synthesizer_name
0,Can you provide detailed information about the...,[' \n \nPROSPECTBOSS \nPROSPECTBOSS SALES TEA...,The PROSPECTBOSS SALES TEAM can be contacted a...,single_hop_specifc_query_synthesizer
1,What do NEURO-EMOTIONAL PERSUATION QUESTIONS d...,"["" \n \n 2 \n \nWHAT NEPQ (NEURO-EMOTIONAL PE...",NEURO-EMOTIONAL PERSUATION QUESTIONS help gath...,single_hop_specifc_query_synthesizer
2,How do your prospects benefit from engaging in...,"["" \n \n 3 \n \nA good doctor follows a simil...",Your prospects benefit from engaging in strate...,single_hop_specifc_query_synthesizer
3,What does the concept of Level 8 imply about t...,[' \n \n 5 \n \ninitiate a conversation by as...,"Level 8 refers to the process where prospects,...",single_hop_specifc_query_synthesizer
4,How do you become a trusted authority?,"[' \n \n 6 \n \n \nSo, later in the sales con...",You become a trusted authority by listening to...,single_hop_specifc_query_synthesizer


In [31]:
len(df_cleaned)

100

In [32]:
# Select only the 'question' and 'ground_truth' columns
df_selected = df_cleaned[['user_input', 'reference_contexts', 'reference']]

# Display the selected columns
df_selected.head()

,user_input,reference_contexts,reference
0,Can you provide detailed information about the...,[' \n \nPROSPECTBOSS \nPROSPECTBOSS SALES TEA...,The PROSPECTBOSS SALES TEAM can be contacted a...
1,What do NEURO-EMOTIONAL PERSUATION QUESTIONS d...,"["" \n \n 2 \n \nWHAT NEPQ (NEURO-EMOTIONAL PE...",NEURO-EMOTIONAL PERSUATION QUESTIONS help gath...
2,How do your prospects benefit from engaging in...,"["" \n \n 3 \n \nA good doctor follows a simil...",Your prospects benefit from engaging in strate...
3,What does the concept of Level 8 imply about t...,[' \n \n 5 \n \ninitiate a conversation by as...,"Level 8 refers to the process where prospects,..."
4,How do you become a trusted authority?,"[' \n \n 6 \n \n \nSo, later in the sales con...",You become a trusted authority by listening to...


In [33]:
# Filter out rows where 'ground_truth' contains the unwanted phrase
df_filtered = df_selected[~df_selected['reference'].str.contains("The answer to given question is not present in| As an AI|The context does not provide|I'm sorry,|The text doesn't provide information")]

# Display the filtered DataFrame
df_filtered.head()

,user_input,reference_contexts,reference
0,Can you provide detailed information about the...,[' \n \nPROSPECTBOSS \nPROSPECTBOSS SALES TEA...,The PROSPECTBOSS SALES TEAM can be contacted a...
1,What do NEURO-EMOTIONAL PERSUATION QUESTIONS d...,"["" \n \n 2 \n \nWHAT NEPQ (NEURO-EMOTIONAL PE...",NEURO-EMOTIONAL PERSUATION QUESTIONS help gath...
2,How do your prospects benefit from engaging in...,"["" \n \n 3 \n \nA good doctor follows a simil...",Your prospects benefit from engaging in strate...
3,What does the concept of Level 8 imply about t...,[' \n \n 5 \n \ninitiate a conversation by as...,"Level 8 refers to the process where prospects,..."
4,How do you become a trusted authority?,"[' \n \n 6 \n \n \nSo, later in the sales con...",You become a trusted authority by listening to...


In [34]:
len(df_filtered)

99

In [35]:
df_filtered.to_csv(f"./dataset_filtered.csv", index=False)


### using trulens

In [16]:
import os

if "OPENAI_API_KEY" not in os.environ:
    print("No openaai key found.")
print("OpenAI key found.")

OpenAI key found.


In [17]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [18]:
import numpy as np
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

# Initialize provider class
provider = OpenAI(model_engine="gpt-4.1-mini")

# select context to be used in feedback. the location of context is app specific.

context = TruLlama.select_context(query_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


instrument app for logging with Trulens

In [19]:
tru_query_engine_recorder = TruLlama(
    query_engine,
    app_name="LlamaIndex_App",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'llama_index_instrumentation.DispatcherSpanMixin'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for base <class 'abc.ABC'>
instrumenting <class 'llama_index.embeddings.openai.base.OpenAIEmbedding'> for